# Assignment 3: Train your own LLMs
### **Course Name:** CSC4100 Natural Language Processing




## Preliminary Preparation

Before proceeding with model training, ensure your environment is properly configured by following these steps:

1. Install the necessary Python packages.
2. Import the required libraries.

In [ ]:
!pip install -q -U \
  "transformers>=4.45.0" \
  "accelerate>=1.0.0" \
  "peft>=0.13.0" \
  "datasets>=2.20.0" \
  "sentencepiece" \
  "protobuf<5"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 54.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.


In [ ]:
!nvidia-smi

Sun Nov 16 15:01:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P0             53W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 40.5 MB/s eta 0:00:00



## Load Pre-trained model and tokenizer

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "Qwen/Qwen2.5-3B-Instruct"

# 4-bit config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
)

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# model in 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,  # <- this is the *new* transformers API
    device_map="auto",
)

model.eval()
print("Loaded 4-bit model:", model_id)
print("Device:", next(model.parameters()).device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Loaded 4-bit model: Qwen/Qwen2.5-3B-Instruct
Device: cuda:0


## Preprocess the quantized model for training

---



In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,                      # more capacity
    lora_alpha=32,             # ~2 * r is a good rule of thumb
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    # optional but recommended: focus on attention + MLP projections
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 29,933,568 || all params: 3,115,872,256 || trainable%: 0.9607


In [ ]:
from jinja2 import Template

template = Template(tokenizer.chat_template)
message = "Please introduce yourself"
print(f"message:\n{message}\n")
message_send_to_model=template.render(messages=[{"role": "user", "content": message}],bos_token=tokenizer.bos_token,add_generation_prompt=True)
print(f"message_send_to_model:\n{message_send_to_model}")

message:
Please introduce yourself

message_send_to_model:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Please introduce yourself<|im_end|>
<|im_start|>assistant



## Chat Template Usage



In [ ]:
template = Template(tokenizer.chat_template)
@torch.no_grad()
def generate(prompt):
    modelInput=template.render(messages=[{"role": "user", "content": prompt}],bos_token= tokenizer.bos_token,add_generation_prompt=True)
    print("-"*80)
    print(f"model_input_string:\n{modelInput}")
    input_ids = tokenizer.encode(modelInput, add_special_tokens=False, return_tensors='pt').to("cuda:0")
    outputs = model.generate(input_ids, do_sample=False)
    model_return_string = tokenizer.decode(*outputs, skip_special_tokens=False)
    print("-"*80)
    print(f"model_return_string:\n{model_return_string}")
    generated_ids = outputs[:, input_ids.shape[1]:]
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
    return generated_text

query = "Please introduce yourself"
print("-"*80)
print(f"query:\n{query}")
response = generate(query)
print("-"*80)
print(f"response:\n{response}")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


--------------------------------------------------------------------------------
query:
Please introduce yourself
--------------------------------------------------------------------------------
model_input_string:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Please introduce yourself<|im_end|>
<|im_start|>assistant

--------------------------------------------------------------------------------
model_return_string:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Please introduce yourself<|im_end|>
<|im_start|>assistant
Of course! I'm Qwen, a large language model created by Alibaba Cloud. My primary function
--------------------------------------------------------------------------------
response:
Of course! I'm Qwen, a large language model created by Alibaba Cloud. My primary function


## Data Preparation

In [ ]:
from datasets import load_dataset

# 1) Load Finance-Alpaca
raw = load_dataset("gbharti/finance-alpaca")["train"]

# 2) Convert to your `conversations` format
def to_conversation(example):
    # build a single-turn QA
    instr = example["instruction"]
    inp   = example.get("input", "")
    ans   = example["output"]

    if inp:
        user_text = instr.strip() + "\n\nInput: " + inp.strip()
    else:
        user_text = instr.strip()

    return {
        "conversations": [
            {"from": "human", "value": user_text},
            {"from": "gpt",   "value": ans},
        ]
    }

dataset = raw.map(to_conversation)
print(dataset[0]["conversations"])


README.md:   0%|          | 0.00/831 [00:00<?, ?B/s]

Cleaned_date.json:   0%|          | 0.00/42.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/68912 [00:00<?, ? examples/s]

Map:   0%|          | 0/68912 [00:00<?, ? examples/s]

[{'from': 'human', 'value': 'For a car, what scams can be plotted with 0% financing vs rebate?'}, {'from': 'gpt', 'value': "The car deal makes money 3 ways. If you pay in one lump payment. If the payment is greater than what they paid for the car, plus their expenses, they make a profit. They loan you the money. You make payments over months or years, if the total amount you pay is greater than what they paid for the car, plus their expenses, plus their finance expenses they make money. Of course the money takes years to come in, or they sell your loan to another business to get the money faster but in a smaller amount. You trade in a car and they sell it at a profit. Of course that new transaction could be a lump sum or a loan on the used car... They or course make money if you bring the car back for maintenance, or you buy lots of expensive dealer options. Some dealers wave two deals in front of you: get a 0% interest loan. These tend to be shorter 12 months vs 36,48,60 or even 72 mo

In [ ]:
from torch.utils.data import random_split
train_dataset_size, val_dataset_size = 100, 20
test_dataset_size = 50
train_dataset, val_dataset,test_dataset, _ = random_split(dataset, [train_dataset_size, val_dataset_size, test_dataset_size, len(dataset)-test_dataset_size-train_dataset_size-val_dataset_size])
print(train_dataset[0]['conversations'])

[{'from': 'human', 'value': 'How does giving to charity work?'}, {'from': 'gpt', 'value': 'If something is tax-deductible in the US, it means that, in the eyes of the Internal Revenue Service, you effectively didn\'t earn that money.  Within restrictions, your adjusted gross income, which is the income that your tax is calculated on, is reduced by the amount of your tax deductions.  In the case of the ASPCA, they\'ve jumped through the appropriate hoops to become a 501(c)(3) organization, which, among other things, means that donations to them are tax-deductible by the donor (a) if they itemize, and (b) if they haven\'t reached a donation cap.  That\'s the carrot that encourages donations to these organizations.  There are restrictions, meaning that there can be only certain types of privileges or exchange between the donor and the organization.  Essentially, it has to be a donation, and not a purchase of substantial goods or services.  Your donation to these kinds of organizations doe

In [ ]:
import transformers
from typing import Dict, Sequence, List
from torch.utils.data import Dataset
from dataclasses import dataclass

def preprocess(
    sources,
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    template = Template(tokenizer.chat_template)
    max_seq_len = tokenizer.model_max_length
    messages = []
    for i, source in enumerate(sources):
        if source[0]["from"] != "human":
            # Skip the first one if it is not from human
            source = source[1:]

        for j in range(0, len(source), 2):
            if j+1 >= len(source): continue
            q = source[j]["value"]
            a = source[j+1]["value"]
            assert q is not None and a is not None, f'q:{q} a:{a}'
            input =  template.render(messages=[{"role": "user", "content": q},{"role": "assistant", "content": a}],bos_token=tokenizer.bos_token,add_generation_prompt=False)
            input_ids = tokenizer.encode(input, add_special_tokens= False)

            query = template.render(messages=[{"role": "user", "content": q}],bos_token=tokenizer.bos_token,add_generation_prompt=True)
            query_ids = tokenizer.encode(query, add_special_tokens= False)

            labels = [-100]*len(query_ids) + input_ids[len(query_ids):]
            assert len(labels) == len(input_ids)
            if len(input_ids) == 0: continue
            messages.append({"input_ids": input_ids[-max_seq_len:], "labels": labels[-max_seq_len:]})

    input_ids = [item["input_ids"] for item in messages]
    labels = [item["labels"] for item in messages]

    max_len = max(len(x) for x in input_ids)

    max_len = min(max_len, max_seq_len)
    input_ids = [ item[:max_len] + [tokenizer.eos_token_id]*(max_len-len(item)) for item in input_ids]
    labels = [ item[:max_len] + [-100]*(max_len-len(item)) for item in labels]

    input_ids = torch.LongTensor(input_ids)
    labels = torch.LongTensor(labels)
    return {
        "input_ids": input_ids,
        "labels": labels
    }


class InstructDataset(Dataset):
    def __init__(self, data: Sequence, tokenizer: transformers.PreTrainedTokenizer) -> None:
        super().__init__()
        self.tokenizer = tokenizer
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index) -> Dict[str, torch.Tensor]:
        sources = self.data[index]
        if isinstance(index, int):
            sources = [sources]
        data_dict = preprocess([e['conversations'] for e in sources], self.tokenizer)
        if isinstance(index, int):
            data_dict = dict(input_ids=data_dict["input_ids"][0], labels=data_dict["labels"][0])
        return data_dict


@dataclass
class DataCollatorForSupervisedDataset(object):
    tokenizer: transformers.PreTrainedTokenizer
    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids,
            batch_first=True,
            padding_value=self.tokenizer.pad_token_id)
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [ ]:
train_dataset = InstructDataset(train_dataset, tokenizer)
val_dataset = InstructDataset(val_dataset, tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [ ]:
sample_data = train_dataset[9]
IGNORE_INDEX=-100

print("=" * 80)
print("Debuging: ")
print(f"Input_ids\n{sample_data['input_ids']}")
print(f"Label_ids\n{sample_data['labels']}")
print("-" * 80)
print(f"Input:\n{tokenizer.decode(sample_data['input_ids'])}")
print("-" * 80)
N_id = tokenizer.encode("N", add_special_tokens= False)[0]
print(f"Label:\n{tokenizer.decode([N_id if x == -100 else x for x in sample_data['labels']])}")
print("=" * 80)


Debuging: 
Input_ids
tensor([151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
           553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
            13, 151645,    198, 151644,    872,    198,  49978,    279,   2661,
           738,    304,    311,   1378,  74505,   1741,    429,    279,   6672,
          1948,    862,  36398,    374,  81078,    382,   2505,     25,    314,
            17,     22,     11,    220,     16,     21,     11,    220,     17,
            20,     11,    220,     16,     23,     11,    220,     17,     24,
            92, 151645,    198, 151644,  77091,    198,  70584,    362,    284,
           314,     17,     22,     11,    220,     16,     21,     11,    220,
            16,     23,    532,  70584,    425,    284,    314,     17,     20,
            11,    220,     17,     24,     92, 151645,    198])
Label_ids
tensor([  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100, 

## Training

In [ ]:
training_arguments = transformers.TrainingArguments(
    output_dir="./checkpoints",
    num_train_epochs=3,                     # much better learning
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,          # effective batch = 8 (stable!)
    optim="paged_adamw_32bit",

    learning_rate=2e-5,                     # CRITICAL: main fix for perplexity
    weight_decay=0.0,                       # recommended for LoRA

    warmup_ratio=0.1,                       # smoother warmup
    lr_scheduler_type="linear",             # better for SFT; prevents early LR collapse

    max_steps=-1,
    gradient_checkpointing=True,
    logging_steps=1,
    save_steps=0,
    report_to="none",

    bf16=True,                               # use bf16 on A100/T4 (Colab supports)
)


In [ ]:
model.train()
trainer = transformers.Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator
)
trainer.train()

/tmp/ipython-input-3539358626.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1,2.453400
2,2.369100
3,3.203400
4,2.615700
5,2.876300
6,2.351500
7,2.332600
8,2.538800
9,2.615600
10,2.337500


TrainOutput(global_step=39, training_loss=2.2547138134638467, metrics={'train_runtime': 112.2042, 'train_samples_per_second': 2.674, 'train_steps_per_second': 0.348, 'total_flos': 1064487400439808.0, 'train_loss': 2.2547138134638467, 'epoch': 3.0})

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

model.print_trainable_parameters()

trainable params: 29,933,568 || all params: 3,115,872,256 || trainable%: 0.9607


In [ ]:
import math
!pip install -q -U git+https://github.com/huggingface/accelerate.git
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Perplexity: 6.62


## Save Trained LoRA

In [ ]:
!pwd
output_path = "ilora_csc4100"
trainer.save_model(output_path)

/content


### Test the trained model

In [ ]:
template = Template(tokenizer.chat_template)
@torch.no_grad()
def generate(prompt):
    modelInput = template.render(messages=[{"role": "user", "content": prompt}],bos_token= tokenizer.bos_token,add_generation_prompt=True)
    input_ids = tokenizer.encode(modelInput, add_special_tokens=False, return_tensors='pt').to("cuda:0")
    outputs = model.generate(input_ids, temperature=1.0)
    model_return_string = tokenizer.decode(*outputs, skip_special_tokens=False)
    print("-"*80)
    print(f"model_return_string:\n{model_return_string}")
    generated_ids = outputs[:, input_ids.shape[1]:]
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
    return generated_text

query = "I got hit with a big drawdown in my stock portfolio. What should I do?"
print(f"query:\n{query}")
response = generate(query)
print("-"*80)
print(f"response:\n{response}")

query:
I got hit with a big drawdown in my stock portfolio. What should I do?
--------------------------------------------------------------------------------
model_return_string:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
I got hit with a big drawdown in my stock portfolio. What should I do?<|im_end|>
<|im_start|>assistant
Well, it is not the end of the world and you can still recover your losses. Here's
--------------------------------------------------------------------------------
response:
Well, it is not the end of the world and you can still recover your losses. Here's


In [ ]:
# Empty VRAM
# del model
# del trainer
import gc
import torch
torch.cuda.empty_cache()
gc.collect()
gc.collect()

0

In [ ]:
!nvidia-smi

Sun Nov 16 15:15:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P0             80W /  400W |    5245MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## Load the trained model back and integrate the trained LoRA within.

In [ ]:
from peft import PeftModel

model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map={"":0})
model = PeftModel.from_pretrained(model, output_path)
model = model.merge_and_unload()
model.config.max_length = 512
model.eval()

tokenizer = transformers.AutoTokenizer.from_pretrained(model_id, padding_side="left")
# tokenizer.pad_token = tokenizer.unk_token


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:110: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
@torch.no_grad()
def generate(prompts):
    model_inputs = [template.render(messages=[{"role": "user", "content": prompt}], bos_token=tokenizer.bos_token, add_generation_prompt=True) for prompt in prompts]
    input_ids = tokenizer(model_inputs, add_special_tokens=False, return_tensors='pt', padding=True).to("cuda:0")

    outputs = model.generate(input_ids.input_ids, attention_mask=input_ids.attention_mask, max_new_tokens=100)

    generated_texts = []
    for i in range(len(prompts)):
        generated_ids = outputs[i, input_ids.input_ids.shape[1]:]
        generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        generated_texts.append(generated_text)

    return generated_texts

# test
print("\n\n".join(generate(["Is it better to pay off debt first or invest first?", "Who are you?"])))


Deciding whether to pay off debt first or invest first depends on several factors, including your financial situation, risk tolerance, and goals. Here’s an overview of both approaches:

### Paying Off Debt First
1. **High Interest Rates**: If you have high-interest debts (like credit card debt), paying them down first can save you money in the long run because you avoid accumulating additional interest.
2. **Financial Security**: Debt can create significant stress and financial insecurity. Getting rid of high-interest

I am Qwen, a large language model created by Alibaba Cloud. I'm here to assist with answering questions and generating text across a variety of topics. How can I help you today?


## Test of Alpaca-Finance subset and save output to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install -q evaluate rouge-score


  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -q evaluate

import json
import difflib
from pathlib import Path
from tqdm.auto import tqdm
import evaluate

# Load ROUGE metric
rouge = evaluate.load("rouge")

def string_similarity(a: str, b: str) -> float:
    """Simple character-level similarity."""
    return difflib.SequenceMatcher(None, a, b).ratio()

def build_question(row):
    """Build a single text prompt from instruction + optional input."""
    instr = row["instruction"].strip()
    inp = (row.get("input") or "").strip()
    if inp:
        return instr + "\n\nInput: " + inp
    return instr

def evaluate_on_test_dataset(
    test_dataset,
    out_path: str,
    max_samples: int | None = None,
    batch_size: int = 8,
):
    # Optionally limit number of samples
    if max_samples is not None:
        test_dataset = test_dataset.select(range(min(max_samples, len(test_dataset))))

    preds = []
    refs = []

    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    with out_path.open("w", encoding="utf-8") as f_out:
        # NOTE: step=batch_size is important
        for i in tqdm(range(0, len(test_dataset), batch_size),
                     desc="Evaluating on test_dataset"):
            batch_rows = [test_dataset[j] for j in range(i, min(i + batch_size, len(test_dataset)))]

            questions  = [build_question(r) for r in batch_rows]
            references = [r["output"].strip() for r in batch_rows]

            # your generate() function from above (returns list of strings)
            answers = generate(questions)

            for q, ref, ans in zip(questions, references, answers):
                preds.append(ans)
                refs.append(ref)
                sim = string_similarity(ans, ref)

                rec = {
                    "question": q,
                    "reference_answer": ref,
                    "model_answer": ans,
                    "similarity": sim,
                }
                f_out.write(json.dumps(rec, ensure_ascii=False) + "\n")

    # Aggregate metrics
    rouge_result = rouge.compute(predictions=preds, references=refs)
    avg_sim = sum(string_similarity(p, r) for p, r in zip(preds, refs)) / len(preds)

    print("\nTest set results:")
    for k, v in rouge_result.items():
        print(f"  {k}: {v:.4f}")
    print(f"  avg_similarity: {avg_sim:.4f}")

    return rouge_result, avg_sim


In [ ]:
OUT_PATH = "/content/drive/MyDrive/qwen_3b_alpaca.jsonl"

test_rouge, test_sim = evaluate_on_test_dataset(
    test_dataset,
    out_path=OUT_PATH,
    max_samples=None,   # or e.g. 50 if you only reserved 50 rows
    batch_size=8,
)

print("Saved results to:", OUT_PATH)


Evaluating on test_dataset:   0%|          | 0/7 [00:00<?, ?it/s]


Test set results:
  rouge1: 0.2684
  rouge2: 0.0827
  rougeL: 0.1849
  rougeLsum: 0.2072
  avg_similarity: 0.1531
Saved results to: /content/drive/MyDrive/qwen_3b_alpaca.jsonl


## Test on Financial Advisor 100 dataset


In [ ]:
from datasets import load_dataset
from tqdm import tqdm
import difflib

import evaluate


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import difflib
from pathlib import Path
from datasets import load_dataset
from tqdm import tqdm
import evaluate

# -------------------
# 1) Config
# -------------------
DATASET_NAME = "nihiluis/financial-advisor-100"
SPLIT        = "train"
N_SAMPLES    = 50          # change this if you want
BATCH_SIZE   = 16
OUT_PATH = Path("/content/drive/MyDrive/finance_eval_results_3b.jsonl")

# -------------------
# 2) Load dataset and convert to plain list of dicts
# -------------------
ds = load_dataset(DATASET_NAME, split=SPLIT)
N_SAMPLES = min(N_SAMPLES, len(ds))

# This is the key: make a plain Python list so every element is a dict
ds_list = ds.select(range(N_SAMPLES)).to_list()

def build_prompt(question: str) -> str:
    return f"""You are a certified financial advisor.

Provide a concise, accurate answer to the following financial question:

Question: {question}

Answer:"""

# -------------------
# 3) Run batched generation
# -------------------
all_results = []

for start in tqdm(range(0, N_SAMPLES, BATCH_SIZE), desc="Evaluating"):
    end = min(start + BATCH_SIZE, N_SAMPLES)
    batch = ds_list[start:end]           # each item is a dict: {"question": ..., "answer": ...}

    questions = [ex["question"] for ex in batch]
    gts       = [ex["answer"]   for ex in batch]
    prompts   = [build_prompt(q) for q in questions]

    preds = generate(prompts)            # your generate() from before
    if isinstance(preds, str):
        preds = [preds]

    for q, gt, pred in zip(questions, gts, preds):
        rec = {
            "question": q,
            "ground_truth": gt.strip(),
            "model_answer": pred.strip(),
        }
        all_results.append(rec)

# -------------------
# 4) Save to JSONL
# -------------------
with OUT_PATH.open("w", encoding="utf-8") as f:
    for r in all_results:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"Saved {len(all_results)} examples to {OUT_PATH}")

# -------------------
# 5) String similarity
# -------------------
def sim(a, b):
    return difflib.SequenceMatcher(None, a.lower(), b.lower()).ratio()

refs  = [r["ground_truth"] for r in all_results]
preds = [r["model_answer"] for r in all_results]

scores = [sim(p, g) for p, g in zip(preds, refs)]
avg_sim = sum(scores) / len(scores)
print(f"Avg string similarity: {avg_sim:.3f}")

for i, s in enumerate(scores[:10]):
    print(f"Example {i+1}: sim={s:.3f}")

# -------------------
# 6) ROUGE
# -------------------
rouge = evaluate.load("rouge")
rouge_res = rouge.compute(predictions=preds, references=refs)
print("ROUGE:", rouge_res)


README.md:   0%|          | 0.00/539 [00:00<?, ?B/s]

data/train-00000-of-00001-f0708e72202ddc(…):   0%|          | 0.00/321k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

Evaluating: 100%|██████████| 4/4 [01:36<00:00, 24.11s/it]


Saved 50 examples to /content/drive/MyDrive/finance_eval_results_3b.jsonl
Avg string similarity: 0.049
Example 1: sim=0.031
Example 2: sim=0.051
Example 3: sim=0.027
Example 4: sim=0.083
Example 5: sim=0.044
Example 6: sim=0.020
Example 7: sim=0.016
Example 8: sim=0.198
Example 9: sim=0.066
Example 10: sim=0.029
ROUGE: {'rouge1': np.float64(0.2618928128579842), 'rouge2': np.float64(0.07617238903358317), 'rougeL': np.float64(0.13700024847174175), 'rougeLsum': np.float64(0.2138689315562609)}
